# Group Emotion Recognition

## 1. Data Preprocessing

In [1]:
import cv2
import os
from PIL import Image
import glob
import numpy as np

### 1.1 Face and Label Detection

### 1.1.1 Using Google Vision API

**Import Google Vision Library**

In [ ]:
from google.cloud import vision

**Implement Face Detection function**

In [ ]:
def detect_face_google(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = vision.types.Image(content=content)

    return client.face_detection(image=image).face_annotations

**Implement Label Detection function**

In [ ]:
def detect_labels_google(path):
    """Detects labels in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        print(label.description)
    
    labels_list = [label.description for label in labels]
    
    return labels_list

In [ ]:
def crop_faces(image_file, cropped_images_path, faces):
    count = 1
    image = Image.open(image_file)
    
#     print(len(faces))

    if not faces:
        print("No face detected in the image.")
        return
    
    for face in faces:
        coordinates = [(vertex.x, vertex.y)
            for vertex in face.bounding_poly.vertices]
        
        x_coordinates, y_coordinates = [], []
        for vertex in face.bounding_poly.vertices:
            x_coordinates.append(vertex.x)
            y_coordinates.append(vertex.y)
        
        x0, x1, y0, y1 = x_coordinates[0], x_coordinates[2], y_coordinates[0], y_coordinates[2]
        
        box = (x0, y0, x1, y1)
        cropped_image = image.crop(box)
                                                              
        image_name = (image_file.split("/")[-1])[:-4]
        cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
        count+=1

In [ ]:
def resize_faces_google(cropped_images_path, scaled_images_path, size):
    count = 1
    for file in glob.glob(cropped_images_path+"*.jpg"):
        image = cv2.imread(file)
        height, width = image.shape[:2]
        
        height_ratio, width_ratio = float(size/height), float(size/width)

        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
#         print(file)
        image_name = (file.split("/")[-1])
        cv2.imwrite(scaled_images_path + image_name, resized)

In [ ]:
images = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

for image_category in images:
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        with open(image_file, 'rb') as image:
            faces = detect_face_google(image)

            # Reset the file pointer, so we can read the file again
            image.seek(0)
            crop_faces(image_file, image_category[1], faces)
        break
    break

    resize_faces_google(image_category[1], image_category[2], 64)


---

### 1.1.2. Using OpenCV DNN

In [ ]:
# load our serialized model from disk
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt',
                               'res10_300x300_ssd_iter_140000.caffemodel'
                               )

In [ ]:
def extract_faces_cv(image_file, cropped_images_path):
    # load the input image and construct an input blob for the image
    # by resizing to a fixed 300x300 pixels and then normalizing it
    image = cv2.imread(image_file)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300,
                                 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    faces = net.forward()
    count = 1
    image = Image.open(image_file)
    
    if faces is None:
        print("No face detected in the image.")
        return

    # loop over the faces
    for i in range(faces.shape[2]):

        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = faces[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.5:

            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            cropped_image = image.crop(box)

            image_name = (image_file.split("/")[-1])[:-4]
            cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
            count+=1

In [ ]:
def resize_faces_cv(cropped_images_path, scaled_images_path, size):
    count = 1
    for file in glob.glob(cropped_images_path+"*.jpg"):
        image = cv2.imread(file)
        height, width = image.shape[:2]
        
        height_ratio, width_ratio = float(size/height), float(size/width)

        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
#         print(file)
        image_name = (file.split("/")[-1])
        cv2.imwrite(scaled_images_path + image_name, resized)

In [ ]:
images = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

for image_category in images:
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        extract_faces_cv(image_file, image_category[1])
        break
    resize_faces_cv(image_category[1], image_category[2], 64)

---

In [2]:
import dlib

In [20]:
# dlib hog + svm based face detector
detector = dlib.get_frontal_face_detector()

def extract_faces(image_file, cropped_images_path):
    # load input image
    image = cv2.imread(image_file)
    count = 1
    image_height, image_width = image.shape[:2]
    
    if image is None:
        print("Could not read input image")
        exit()
    
    # apply face detection
    faces = detector(image, 1)
    
    # loop over detected faces
    for face in faces:
        cropped_image = image[max(0, face.top()): min(face.bottom(), image_height),
                    max(0, face.left()): min(face.right(), image_width)]
        image_name = (image_file.split("/")[-1])[:-4]
        cv2.imwrite(cropped_images_path + image_name + "_face_" + str(count) + ".jpg", cropped_image)
        count+=1

In [21]:
def resize_faces(cropped_images_path, scaled_images_path, size):
    count = 1
    for file in glob.glob(cropped_images_path+"*.jpg"):
        image = cv2.imread(file)
        height, width = image.shape[:2]
        
        height_ratio, width_ratio = float(size/height), float(size/width)

        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
        image_name = (file.split("/")[-1])
        cv2.imwrite(scaled_images_path + image_name, resized)

In [22]:
images = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

for image_category in images:
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        extract_faces(image_file, image_category[1])
    resize_faces(image_category[1], image_category[2], 64)

emotiw/train/Positive/pos_1.jpg
emotiw/train/Positive/pos_10.jpg
emotiw/train/Positive/pos_100.jpg
emotiw/train/Positive/pos_1000.jpg
emotiw/train/Positive/pos_1001.jpg
emotiw/train/Positive/pos_1002.jpg
emotiw/train/Positive/pos_1003.jpg
emotiw/train/Positive/pos_1004.jpg
emotiw/train/Positive/pos_1005.jpg
emotiw/train/Positive/pos_1006.jpg
emotiw/train/Positive/pos_1007.jpg
emotiw/train/Positive/pos_1008.jpg
emotiw/train/Positive/pos_1009.jpg
emotiw/train/Positive/pos_101.jpg
emotiw/train/Positive/pos_1010.jpg
emotiw/train/Positive/pos_1011.jpg
emotiw/train/Positive/pos_1012.jpg
emotiw/train/Positive/pos_1013.jpg
emotiw/train/Positive/pos_1014.jpg
emotiw/train/Positive/pos_1015.jpg
emotiw/train/Positive/pos_1016.jpg
emotiw/train/Positive/pos_1017.jpg
emotiw/train/Positive/pos_1018.jpg
emotiw/train/Positive/pos_1019.jpg
emotiw/train/Positive/pos_102.jpg
emotiw/train/Positive/pos_1020.jpg
emotiw/train/Positive/pos_1021.jpg
emotiw/train/Positive/pos_1022.jpg
emotiw/train/Positive/pos_10

emotiw/train/Positive/pos_121.jpg
emotiw/train/Positive/pos_1210.jpg
emotiw/train/Positive/pos_1211.jpg
emotiw/train/Positive/pos_1212.jpg
emotiw/train/Positive/pos_1213.jpg
emotiw/train/Positive/pos_1214.jpg
emotiw/train/Positive/pos_1215.jpg
emotiw/train/Positive/pos_1216.jpg
emotiw/train/Positive/pos_1217.jpg
emotiw/train/Positive/pos_1218.jpg
emotiw/train/Positive/pos_1219.jpg
emotiw/train/Positive/pos_122.jpg
emotiw/train/Positive/pos_1220.jpg
emotiw/train/Positive/pos_1221.jpg
emotiw/train/Positive/pos_1222.jpg
emotiw/train/Positive/pos_1223.jpg
emotiw/train/Positive/pos_1224.jpg
emotiw/train/Positive/pos_1225.jpg
emotiw/train/Positive/pos_1226.jpg
emotiw/train/Positive/pos_1227.jpg
emotiw/train/Positive/pos_1228.jpg
emotiw/train/Positive/pos_1229.jpg
emotiw/train/Positive/pos_123.jpg
emotiw/train/Positive/pos_1230.jpg
emotiw/train/Positive/pos_1231.jpg
emotiw/train/Positive/pos_1232.jpg
emotiw/train/Positive/pos_1233.jpg
emotiw/train/Positive/pos_1234.jpg
emotiw/train/Positive/p

emotiw/train/Positive/pos_1422.jpg
emotiw/train/Positive/pos_1423.jpg
emotiw/train/Positive/pos_1424.jpg
emotiw/train/Positive/pos_1425.jpg
emotiw/train/Positive/pos_1426.jpg
emotiw/train/Positive/pos_1427.jpg
emotiw/train/Positive/pos_1428.jpg
emotiw/train/Positive/pos_1429.jpg
emotiw/train/Positive/pos_143.jpg
emotiw/train/Positive/pos_1430.jpg
emotiw/train/Positive/pos_1431.jpg
emotiw/train/Positive/pos_1432.jpg
emotiw/train/Positive/pos_1433.jpg
emotiw/train/Positive/pos_1434.jpg
emotiw/train/Positive/pos_1435.jpg
emotiw/train/Positive/pos_1436.jpg
emotiw/train/Positive/pos_1437.jpg
emotiw/train/Positive/pos_1438.jpg
emotiw/train/Positive/pos_1439.jpg
emotiw/train/Positive/pos_144.jpg
emotiw/train/Positive/pos_1440.jpg
emotiw/train/Positive/pos_1441.jpg
emotiw/train/Positive/pos_1442.jpg
emotiw/train/Positive/pos_1443.jpg
emotiw/train/Positive/pos_1444.jpg
emotiw/train/Positive/pos_1445.jpg
emotiw/train/Positive/pos_1446.jpg
emotiw/train/Positive/pos_1447.jpg
emotiw/train/Positive/

emotiw/train/Positive/pos_1634.jpg
emotiw/train/Positive/pos_1635.jpg
emotiw/train/Positive/pos_1636.jpg
emotiw/train/Positive/pos_1637.jpg
emotiw/train/Positive/pos_1638.jpg
emotiw/train/Positive/pos_1639.jpg
emotiw/train/Positive/pos_164.jpg
emotiw/train/Positive/pos_1640.jpg
emotiw/train/Positive/pos_1641.jpg
emotiw/train/Positive/pos_1642.jpg
emotiw/train/Positive/pos_1643.jpg
emotiw/train/Positive/pos_1644.jpg
emotiw/train/Positive/pos_1645.jpg
emotiw/train/Positive/pos_1646.jpg
emotiw/train/Positive/pos_1647.jpg
emotiw/train/Positive/pos_1648.jpg
emotiw/train/Positive/pos_1649.jpg
emotiw/train/Positive/pos_165.jpg
emotiw/train/Positive/pos_1650.jpg
emotiw/train/Positive/pos_1651.jpg
emotiw/train/Positive/pos_1652.jpg
emotiw/train/Positive/pos_1653.jpg
emotiw/train/Positive/pos_1654.jpg
emotiw/train/Positive/pos_1655.jpg
emotiw/train/Positive/pos_1656.jpg
emotiw/train/Positive/pos_1657.jpg
emotiw/train/Positive/pos_1658.jpg
emotiw/train/Positive/pos_1659.jpg
emotiw/train/Positive/

emotiw/train/Positive/pos_1847.jpg
emotiw/train/Positive/pos_1848.jpg
emotiw/train/Positive/pos_1849.jpg
emotiw/train/Positive/pos_185.jpg
emotiw/train/Positive/pos_1850.jpg
emotiw/train/Positive/pos_1851.jpg
emotiw/train/Positive/pos_1852.jpg
emotiw/train/Positive/pos_1853.jpg
emotiw/train/Positive/pos_1854.jpg
emotiw/train/Positive/pos_1855.jpg
emotiw/train/Positive/pos_1856.jpg
emotiw/train/Positive/pos_1857.jpg
emotiw/train/Positive/pos_1858.jpg
emotiw/train/Positive/pos_1859.jpg
emotiw/train/Positive/pos_186.jpg
emotiw/train/Positive/pos_1860.jpg
emotiw/train/Positive/pos_1861.jpg
emotiw/train/Positive/pos_1862.jpg
emotiw/train/Positive/pos_1863.jpg
emotiw/train/Positive/pos_1864.jpg
emotiw/train/Positive/pos_1865.jpg
emotiw/train/Positive/pos_1866.jpg
emotiw/train/Positive/pos_1867.jpg
emotiw/train/Positive/pos_1868.jpg
emotiw/train/Positive/pos_1869.jpg
emotiw/train/Positive/pos_187.jpg
emotiw/train/Positive/pos_1870.jpg
emotiw/train/Positive/pos_1871.jpg
emotiw/train/Positive/p

...

emotiw/train/Negative/neg_2417.jpg
emotiw/train/Negative/neg_2418.jpg
emotiw/train/Negative/neg_2419.jpg
emotiw/train/Negative/neg_242.jpg
emotiw/train/Negative/neg_2420.jpg
emotiw/train/Negative/neg_2421.jpg
emotiw/train/Negative/neg_2422.jpg
emotiw/train/Negative/neg_2423.jpg
emotiw/train/Negative/neg_2424.jpg
emotiw/train/Negative/neg_2425.jpg
emotiw/train/Negative/neg_2426.jpg
emotiw/train/Negative/neg_2427.jpg
emotiw/train/Negative/neg_2428.jpg
emotiw/train/Negative/neg_2429.jpg
emotiw/train/Negative/neg_243.jpg
emotiw/train/Negative/neg_2430.jpg
emotiw/train/Negative/neg_2431.jpg
emotiw/train/Negative/neg_2432.jpg
emotiw/train/Negative/neg_2433.jpg
emotiw/train/Negative/neg_2434.jpg
emotiw/train/Negative/neg_2435.jpg
emotiw/train/Negative/neg_2436.jpg
emotiw/train/Negative/neg_2437.jpg
emotiw/train/Negative/neg_2438.jpg
emotiw/train/Negative/neg_2439.jpg
emotiw/train/Negative/neg_244.jpg
emotiw/train/Negative/neg_2440.jpg
emotiw/train/Negative/neg_2441.jpg
emotiw/train/Negative/n

emotiw/train/Negative/neg_2629.jpg
emotiw/train/Negative/neg_263.jpg
emotiw/train/Negative/neg_2630.jpg
emotiw/train/Negative/neg_2631.jpg
emotiw/train/Negative/neg_2632.jpg
emotiw/train/Negative/neg_2633.jpg
emotiw/train/Negative/neg_2634.jpg
emotiw/train/Negative/neg_2635.jpg
emotiw/train/Negative/neg_2636.jpg
emotiw/train/Negative/neg_2637.jpg
emotiw/train/Negative/neg_2638.jpg
emotiw/train/Negative/neg_2639.jpg
emotiw/train/Negative/neg_264.jpg
emotiw/train/Negative/neg_2640.jpg
emotiw/train/Negative/neg_2641.jpg
emotiw/train/Negative/neg_2642.jpg
emotiw/train/Negative/neg_2643.jpg
emotiw/train/Negative/neg_2644.jpg
emotiw/train/Negative/neg_2645.jpg
emotiw/train/Negative/neg_2646.jpg
emotiw/train/Negative/neg_2647.jpg
emotiw/train/Negative/neg_2648.jpg
emotiw/train/Negative/neg_2649.jpg
emotiw/train/Negative/neg_265.jpg
emotiw/train/Negative/neg_2650.jpg
emotiw/train/Negative/neg_2651.jpg
emotiw/train/Negative/neg_2652.jpg
emotiw/train/Negative/neg_2653.jpg
emotiw/train/Negative/n

emotiw/train/Negative/neg_360.jpg
emotiw/train/Negative/neg_361.jpg
emotiw/train/Negative/neg_362.jpg
emotiw/train/Negative/neg_363.jpg
emotiw/train/Negative/neg_364.jpg
emotiw/train/Negative/neg_365.jpg
emotiw/train/Negative/neg_366.jpg
emotiw/train/Negative/neg_367.jpg
emotiw/train/Negative/neg_368.jpg
emotiw/train/Negative/neg_369.jpg
emotiw/train/Negative/neg_37.jpg
emotiw/train/Negative/neg_370.jpg
emotiw/train/Negative/neg_371.jpg
emotiw/train/Negative/neg_372.jpg
emotiw/train/Negative/neg_373.jpg
emotiw/train/Negative/neg_374.jpg
emotiw/train/Negative/neg_375.jpg
emotiw/train/Negative/neg_376.jpg
emotiw/train/Negative/neg_377.jpg
emotiw/train/Negative/neg_378.jpg
emotiw/train/Negative/neg_379.jpg
emotiw/train/Negative/neg_38.jpg
emotiw/train/Negative/neg_380.jpg
emotiw/train/Negative/neg_381.jpg
emotiw/train/Negative/neg_382.jpg
emotiw/train/Negative/neg_383.jpg
emotiw/train/Negative/neg_384.jpg
emotiw/train/Negative/neg_385.jpg
emotiw/train/Negative/neg_386.jpg
emotiw/train/Neg

emotiw/train/Negative/neg_579.jpg
emotiw/train/Negative/neg_58.jpg
emotiw/train/Negative/neg_580.jpg
emotiw/train/Negative/neg_581.jpg
emotiw/train/Negative/neg_582.jpg
emotiw/train/Negative/neg_583.jpg
emotiw/train/Negative/neg_584.jpg
emotiw/train/Negative/neg_585.jpg
emotiw/train/Negative/neg_586.jpg
emotiw/train/Negative/neg_587.jpg
emotiw/train/Negative/neg_588.jpg
emotiw/train/Negative/neg_589.jpg
emotiw/train/Negative/neg_59.jpg
emotiw/train/Negative/neg_590.jpg
emotiw/train/Negative/neg_591.jpg
emotiw/train/Negative/neg_592.jpg
emotiw/train/Negative/neg_593.jpg
emotiw/train/Negative/neg_594.jpg
emotiw/train/Negative/neg_595.jpg
emotiw/train/Negative/neg_596.jpg
emotiw/train/Negative/neg_597.jpg
emotiw/train/Negative/neg_598.jpg
emotiw/train/Negative/neg_599.jpg
emotiw/train/Negative/neg_6.jpg
emotiw/train/Negative/neg_60.jpg
emotiw/train/Negative/neg_600.jpg
emotiw/train/Negative/neg_601.jpg
emotiw/train/Negative/neg_602.jpg
emotiw/train/Negative/neg_603.jpg
emotiw/train/Negati

emotiw/train/Negative/neg_798.jpg
emotiw/train/Negative/neg_799.jpg
emotiw/train/Negative/neg_8.jpg
emotiw/train/Negative/neg_80.jpg
emotiw/train/Negative/neg_800.jpg
emotiw/train/Negative/neg_801.jpg
emotiw/train/Negative/neg_802.jpg
emotiw/train/Negative/neg_803.jpg
emotiw/train/Negative/neg_804.jpg
emotiw/train/Negative/neg_805.jpg
emotiw/train/Negative/neg_806.jpg
emotiw/train/Negative/neg_807.jpg
emotiw/train/Negative/neg_808.jpg
emotiw/train/Negative/neg_809.jpg
emotiw/train/Negative/neg_81.jpg
emotiw/train/Negative/neg_810.jpg
emotiw/train/Negative/neg_811.jpg
emotiw/train/Negative/neg_812.jpg
emotiw/train/Negative/neg_813.jpg
emotiw/train/Negative/neg_814.jpg
emotiw/train/Negative/neg_815.jpg
emotiw/train/Negative/neg_816.jpg
emotiw/train/Negative/neg_817.jpg
emotiw/train/Negative/neg_818.jpg
emotiw/train/Negative/neg_819.jpg
emotiw/train/Negative/neg_82.jpg
emotiw/train/Negative/neg_820.jpg
emotiw/train/Negative/neg_821.jpg
emotiw/train/Negative/neg_822.jpg
emotiw/train/Negati

---